## Risks and Assumptions

**May not be able to fetch Geocodes for all HDB flats**

I have tried using Google Map and Nominatim to get geocodes of HDB flats and points of interest. 

Google Map is able to get geocodes for all physical addresses and but Google charges USD7 per 1000 requests for the first 100,000 requests. 

Nominatim is FREE but is unable to get geocodes for about 15% of the addresses. In order to mitigate this risk I will collect more HDB resale transaction data and drop those observations that it failed to get geocodes. Alternatively I will pay for Google service, use manual method or explore other free geocoding services. 

## Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import geopy.geocoders
from geopy.geocoders import Nominatim, GoogleV3, Bing
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderServiceError
from geopy.exc import GeopyError
import scipy.stats as sps
import seaborn as sns
import time
import datetime
import matplotlib as plt
from bs4 import BeautifulSoup
import requests
import json
import re

# File organization
 - notebook in codes folder
 - data files to be imported in datasets/input folder
 - data files exported in datasets/output folder

## Data Collection, Cleaning and Munging

Types of Data collected

1.HDB Flat Resale transactions (2015 to 2020)
- flat attributes and prices

2.Supply and Demand Factors
- Sales of new HDB flats
- Sales of new private homes
- Number of married people
- Number of Residents

3.Macroeconomic factors
- Consumer Price Index
- Purchasing Manager Index
- Composite Leading Index
- GDP Growth
- CPF interest rates
- Singapore Interbank Offered Rate (SIBOR)
- umemployment rate
- median income of residents
- HDB flat price index
- Private property price index

4.Points of Interest
- Shopping Malls (more malls to be manually added)
- Nature Parks
- Columbaria/crematoria/cemeteries
- Schools
- Sports Facitilites
- MRT/LRT stations
- Hawker centres and markets
- Librarues

## 1. HDB Flat Resale Transactions

Souce: data.gov.sg<br/>
Observation: per sale transaction


Year and Month are based on the dates of registration for the resale transactions

In [3]:
hdb_2017_2020 = pd.read_csv('../datasets/input/resale-flat-prices-2017-2020.csv')
hdb_2015_2016 = pd.read_csv('../datasets/input/resale-flat-prices-2015-2016.csv')

In [185]:
# check for any null value
hdb_2017_2020.isna().sum().any()

False

In [186]:
hdb_2015_2016.isna().sum().any()

False

In [16]:
hdb_2017_2020['month'] = pd.to_datetime(hdb_2017_2020['month'])
hdb_2015_2016['month'] = pd.to_datetime(hdb_2015_2016['month'])

In [187]:
# check duplicated rows
df = pd.DataFrame()
df['duplicate'] = hdb_2017_2020.duplicated()
df[df.duplicate==True]

,duplicate
243,True
305,True
510,True
591,True
672,True
...,...
64397,True
65871,True
69191,True
69836,True


In [188]:
# check duplicated rows
df = pd.DataFrame()
df['duplicate'] = hdb_2015_2016.duplicated()
df[df.duplicate==True]

,duplicate
661,True
2166,True
3896,True
4246,True
5493,True
6076,True
7091,True
7182,True
9114,True
10215,True


In [19]:
# remove duplicated rows
hdb_2015_2016.drop_duplicates(inplace=True)
hdb_2017_2020.drop_duplicates(inplace=True)

In [190]:
hdb_2015_2016.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')

In [191]:
hdb_2017_2020.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')

In [ ]:
# convert column to numberic by extracting the year for hdb_2017_2020
# The same column for hdb_2015_2016 is already numberic so it does not required this step

hdb_2017_2020['remaining_lease'] = (
hdb_2017_2020['remaining_lease'].str.extract(r'(\d+)\s?y?e?a?r?').iloc[:,0].astype(np.int32))

In [27]:
hdb_df = pd.concat([hdb_2015_2016, hdb_2017_2020], ignore_index=True)

In [9]:
hdb_df.shape

(107491, 11)

In [22]:
hdb_df.dtypes

month                  datetime64[ns]
town                           object
flat_type                      object
block                          object
street_name                    object
storey_range                   object
floor_area_sqm                float64
flat_model                     object
lease_commence_date             int64
remaining_lease                 int64
resale_price                  float64
dtype: object

In [23]:
hdb_df.month.dt.year

0         2015
1         2015
2         2015
3         2015
4         2015
          ... 
107463    2020
107464    2020
107465    2020
107466    2020
107467    2020
Name: month, Length: 107468, dtype: int64

In [28]:
# Change all text values to lowercase
hdb_df['town'] = hdb_df['town'].str.lower()
hdb_df['flat_type'] = hdb_df['flat_type'].str.lower()
hdb_df['block'] = hdb_df['block'].str.lower()
hdb_df['street_name'] = hdb_df['street_name'].str.lower()
hdb_df['storey_range'] = hdb_df['storey_range'].str.lower()
hdb_df['flat_model'] = hdb_df['flat_model'].str.lower()

In [29]:
hdb_df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price'],
      dtype='object')

In [30]:
# combine block and street_name to create the address column, and drop the original columns
hdb_df['address'] = hdb_df['block'] + ' ' + hdb_df['street_name']
hdb_df.drop(['block','street_name'],axis=1,inplace=True)

In [31]:
# split date column to year and month
# hdb_df[['year','month']] = hdb_df['month'].str.split("-", 1, expand=True)

# cast year and month to integer
#hdb_df['year'] = hdb_df['year'].astype(np.int32)
#hdb_df['month'] = hdb_df['month'].astype(np.int32)

# convert column to numeric by fixing the range to the starting storey
hdb_df['floor_range'] = hdb_df['storey_range'].str.extract(r'(\d+)\sto').iloc[:,0].astype(np.int32)

# replace spaces and "/" in values to "_"
hdb_df['town'].replace({" ":"_","/":"_"},regex=True,inplace=True)
hdb_df['flat_type'].replace({" ":"_","-":"_"},regex=True,inplace=True)
hdb_df['flat_model'].replace({" ":"_","-":"_"},regex=True,inplace=True)

# convert categorical variables to one-hot encoding
dummy = pd.get_dummies(hdb_df['town'],drop_first=True)
hdb_df = hdb_df.merge(dummy,left_index=True,right_index=True)
#hdb_df.drop(['town'],axis=1,inplace=True)

dummy = pd.get_dummies(hdb_df['flat_model'],prefix="model",drop_first=True)
hdb_df = hdb_df.merge(dummy,left_index=True,right_index=True)
#hdb_df.drop(['flat_model'],axis=1,inplace=True)

dummy = pd.get_dummies(hdb_df['flat_type'],prefix="type",drop_first=True)
hdb_df = hdb_df.merge(dummy,left_index=True,right_index=True)
#hdb_df.drop(['flat_type'],axis=1,inplace=True)

In [33]:
hdb_df.head()

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,...,model_standard,model_terrace,model_type_s1,model_type_s2,type_2_room,type_3_room,type_4_room,type_5_room,type_executive,type_multi_generation
0,2015-01-01,ang_mo_kio,3_room,07 to 09,60.0,improved,1986,70,255000.0,174 ang mo kio ave 4,...,0,0,0,0,0,1,0,0,0,0
1,2015-01-01,ang_mo_kio,3_room,01 to 03,68.0,new_generation,1981,65,275000.0,541 ang mo kio ave 10,...,0,0,0,0,0,1,0,0,0,0
2,2015-01-01,ang_mo_kio,3_room,01 to 03,69.0,new_generation,1980,64,285000.0,163 ang mo kio ave 4,...,0,0,0,0,0,1,0,0,0,0
3,2015-01-01,ang_mo_kio,3_room,01 to 03,68.0,new_generation,1979,63,290000.0,446 ang mo kio ave 10,...,0,0,0,0,0,1,0,0,0,0
4,2015-01-01,ang_mo_kio,3_room,07 to 09,68.0,new_generation,1980,64,290000.0,557 ang mo kio ave 10,...,0,0,0,0,0,1,0,0,0,0


In [34]:
# Many addresses in Singapore use abbreviations, which can affect the ability of geocoding services to get geocodes
# The following will convert abbreviations in addresses to their original forms

hdb_df.replace(regex={
        r"\b(upp)\b":"upper",r"\b(rd)\b":"road",r"\b(lor)\b":"lorong",
        r"\b(ave)\b":"avenue",r"\b(jln)\b":"jalan",r"\b(sth)\b":"south",r"\b(nth)\b":"north",
        r"\b(ctrl)\b":"central",r"\b(blk)\b":"block",r"\b(blvd)\b":"boluevard",
        r"\b(bt)\b":"bukit",r"\b(c'wealth)\b":"commonwealth",r"\b(cl)\b":"close",r"\b(cplx)\b":"complex",
        r"\b(ctr)\b":"centre",r"\b(dr)\b":"drive",r"\b(est)\b":"estate",r"\b(gdn)\b":"garden",
        r"\b(gdns)\b":"gardens",r"\b(gr)\b":"grove",r"\b(hse)\b":"house",r"\b(hts)\b":"heights",
        r"\b(ind)\b":"industrial",r"\b(distripk)\b":"distripark",r"\b(intl)\b":"international",
        r"\b(lk)\b":"link",r"\b(mkt)\b":"market",r"\b(mjd)\b":"masjid",r"\b(mt)\b":"mount",
        r"\b(natl)\b":"national",r"\b(opp)\b":"opposite",r"\b(pk)\b":"park",r"\b(pl)\b":"place",
        r"\b(pt)\b":"point",r"\b(resvr)\b":"reservoir",r"\b(sch)\b":"school",r"\b(sci)\b":"science",
        r"\b(sq)\b":"square",r"\b(ter)\b":"terrace",r"\b(tg)\b":"tanjong",
        r"\b(tp)\b":"temple",r"\b(twr)\b":"tower",r"\b(w'lands)\b":"woodlands",r"\b(wk)\b":"walk",
        r"\b(wtr)\b":"water",r"\b(v)\b":"village",r"\b(veh)\b":"vehicle",r"\b(warehse)\b":"warehouse",
        r"\b(bef)\b":"before",r"\b(aft)\b":"after",r"\b(svc)\b":"service",
        r"\b(svcs)\b":"services",r"\b(sg)\b":"sungei",r"\b(kg)\b":"kampong"}, inplace=True)

In [36]:
hdb_df.rename(columns={"month":"sale_date"},inplace=True)

In [51]:
hdb_df.columns

Index(['sale_date', 'town', 'flat_type', 'storey_range', 'floor_area_sqm',
       'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price',
       'address', 'floor_range', 'bedok', 'bishan', 'bukit_batok',
       'bukit_merah', 'bukit_panjang', 'bukit_timah', 'central_area',
       'choa_chu_kang', 'clementi', 'geylang', 'hougang', 'jurong_east',
       'jurong_west', 'kallang_whampoa', 'marine_parade', 'pasir_ris',
       'punggol', 'queenstown', 'sembawang', 'sengkang', 'serangoon',
       'tampines', 'toa_payoh', 'woodlands', 'yishun', 'model_adjoined_flat',
       'model_apartment', 'model_dbss', 'model_improved',
       'model_improved_maisonette', 'model_maisonette', 'model_model_a',
       'model_model_a2', 'model_model_a_maisonette', 'model_multi_generation',
       'model_new_generation', 'model_premium_apartment',
       'model_premium_apartment_loft', 'model_premium_maisonette',
       'model_simplified', 'model_standard', 'model_terrace', 'model_type_s1',
   

In [47]:
# breakup by year
hdb_2015 =hdb_df[hdb_df.sale_date.dt.year==2015]
hdb_2016 =hdb_df[hdb_df.sale_date.dt.year==2016]
hdb_2017 =hdb_df[hdb_df.sale_date.dt.year==2017]
hdb_2018 =hdb_df[hdb_df.sale_date.dt.year==2018]
hdb_2019 =hdb_df[hdb_df.sale_date.dt.year==2019]
hdb_2020 =hdb_df[hdb_df.sale_date.dt.year==2020]

In [48]:
# merge transactions from 2015 to 2018 for training and validation
hdb_2015_2018 = pd.concat([hdb_2015,hdb_2016,hdb_2017,hdb_2018],
                     ignore_index=True)

# merge transactions from 2019 and 2020 for testing
hdb_2019_2020 = pd.concat([hdb_2019,hdb_2020],ignore_index=True)

In [49]:
# save the cleaned data from 2015 to 2020 feb (all records)
#hdb_df.to_csv('../datasets/hdb/hdb_resale_2015_2020_cleaned.csv', index = False)

# save the cleaned data from 2015 to 2018 (training and validation)
#hdb_2015_2018.to_csv('../datasets/hdb/hdb_resale_2015_2018_cleaned.csv', index = False)

# save the cleaned data from 2019 to 2020 (testing)
#hdb_2019_2020.to_csv('../datasets/hdb/hdb_resale_2019_2020_cleaned.csv', index = False)

In [2]:
# retreive the cleaned data
hdb_df = pd.read_csv('../datasets/hdb/hdb_resale_2015_2020_cleaned.csv')

## 2. Supply and Demand Factors

In [63]:
# The observations in the datasets are monnthly, quarterly or yearly.
# 
# Function to Expand quarterly figures to spread over the 3 months, to be used for datasets with quarterly records
# input dataframe has columns - year, quarter, figure
# output dataframe has columns - year, month, figure


def qtr_2_mth(df_in):
    df = df_in.copy(deep=True)
    
    size = len(df)
    
    for i in range(0,size*3,3):
        for j in range(0,2):
                
            mth = df.iloc[i+j,1]+1

            replica = pd.DataFrame({"year":int(df.iloc[i+j,0]),"month":mth,df.columns[2]:df.iloc[i+j,2]},index=[i+j+1])

            df = pd.concat([df.iloc[:i+j+1],replica,df.iloc[i+j+1:]]).reset_index(drop=True)
    
    return df

### New HDB Flat Launches

Source: https://www.teoalida.com/singapore/btolist/

BTO (Build to Order) flats are new flats sold directly by HDB to citizens who have not taken advantage of the heavily subsidized prices before. From time to time the government increases the supply of BTO flats to dampen the prices in HDB resale flats.

In [169]:
data = requests.get('https://www.teoalida.com/singapore/btolist').content
soup = BeautifulSoup(data, "html.parser")

flat_list = []

table = soup.find(text="Town name").find_parent("table")
for row in table.find_all("tr")[1:]:
    print([cell.get_text(strip=True) for cell in row.find_all("td")])
    
    if len([cell.get_text(strip=True) for cell in row.find_all("td")]) > 0:
        flat_list.append([cell.get_text(strip=True) for cell in row.find_all("td")])
        

bto_df = pd.DataFrame(flat_list,columns=['town','project','launch_date','completion_date','type_studio',
                              'type_2_room','type_3_room','type_4_room','type_5_room',
                              'type_multi_generation','new_flat_offered'])

bto_df['month'] = 0
bto_df['year'] = 0

for i in range(len(bto_df)):
    s = bto_df.iloc[i,2].split(' ')
    
    if len(s)==3:
        m = s[1].replace('2020Aug','Aug')
        y = s[2]

    elif len(s)==2:
        m = s[0]
        y = s[1]
    
    bto_df.iloc[i,11] = m
    bto_df.iloc[i,12] = int(y)


# change months to numeric form
dic = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
bto_df.month = bto_df.month.map(dic)
bto_df.month = bto_df.month.astype(np.int32)

bto_df.replace('',0,inplace=True)

bto_df.fillna(0)

bto_df.type_studio = bto_df.type_studio.astype(np.int32)
bto_df.type_2_room = bto_df.type_2_room.astype(np.int32)
bto_df.type_3_room = bto_df.type_3_room.astype(np.int32)
bto_df.type_4_room = bto_df.type_4_room.astype(np.int32)
bto_df.type_5_room = bto_df.type_5_room.astype(np.int32)
bto_df.type_multi_generation = bto_df.type_multi_generation.astype(np.int32)
bto_df.new_flat_offered = bto_df.new_flat_offered.astype(np.int32)

[]
[]
['Sembawang', 'Flowing Greenery', 'Apr 2001', 'Cancelled', '', '', '', '210', '200', '', '410']
['Sembawang', 'River Edge', 'Apr 2001', 'Apr 2005', '', '', '', '222', '225', '', '447']
['Sengkang', 'Arcadia', 'Apr 2001', 'Apr 2005', '', '', '', '640', '321', '', '961']
['Sengkang', 'Rivervale Green', 'Apr 2001', 'Apr 2005', '', '', '', '576', '200', '', '776']
['Punggol', 'Edgedale Green', 'Dec 2002', 'Dec 2006', '', '', '', '582', '', '', '582']
['Punggol', 'Periwinkle', 'Dec 2002', 'Dec 2006', '', '', '', '450', '', '', '450']
['Sengkang', 'Coris 1', 'Dec 2002', 'Dec 2006', '', '', '', '434', '', '', '434']
['Punggol', 'Sundial', 'Sep 2003', 'Sep 2007', '', '', '', '425', '', '', '425']
['Punggol', 'Sundial(half relaunched as Nautilus)', 'Sep 2003', 'Cancelled', '', '', '', '478', '', '', '478']
['Sembawang', 'Spring Lodge(relaunched as River Lodge)', 'Sep 2003', 'Cancelled', '', '', '', '432', '', '', '432']
['Sengkang', 'Coris 2', 'Sep 2003', 'Sep 2007', '', '', '', '448', ''

['Sengkang', 'Rivervale Shores', '01 Aug 2017', '2Q2022to 3Q2022', '', '1074', '174', '678', '574', '', '2500']
['Punggol', 'Northshore Edge', '14 Nov 2017', '4Q2021', '', '', '', '192', '196', '', '388']
['Sengkang', 'Anchorvale Village', '14 Nov 2017', '1Q2022', '', '104', '103', '', '', '', '207']
['Sengkang', 'Fernvale Glades(+ 273 1-rm, 338 2-rm rental)', '14 Nov 2017', '2Q2021to 3Q2021', '', '533', '104', '390', '221', '52', '1300']
['Geylang', 'Eunos Court', '14 Nov 2017', '2Q2022', '', '107', '97', '538', '', '', '742']
['Tampines', 'Tampines GreenCourt', '14 Nov 2017', '4Q2021to 2Q2022', '', '192', '186', '1228', '586', '', '2192']
[]
['Choa Chu Kang', 'Teck Whye View', '06 Feb 2018', '2Q2022', '', '252', '84', '238', '', '', '574']
['Woodlands', 'Woodlands Glade', '06 Feb 2018', '4Q2021', '', '285', '90', '164', '90', '', '629']
['Geylang', 'Ubi Grove', '06 Feb 2018', '4Q2021to 1Q2022', '', '120', '80', '993', '', '', '1193']
['Tampines', 'Tampines GreenDew', '06 Feb 2018', '

In [170]:
bto_df

,town,project,launch_date,completion_date,type_studio,type_2_room,type_3_room,type_4_room,type_5_room,type_multi_generation,new_flat_offered,month,year
0,Sembawang,Flowing Greenery,Apr 2001,Cancelled,0,0,0,210,200,0,410,4,2001
1,Sembawang,River Edge,Apr 2001,Apr 2005,0,0,0,222,225,0,447,4,2001
2,Sengkang,Arcadia,Apr 2001,Apr 2005,0,0,0,640,321,0,961,4,2001
3,Sengkang,Rivervale Green,Apr 2001,Apr 2005,0,0,0,576,200,0,776,4,2001
4,Punggol,Edgedale Green,Dec 2002,Dec 2006,0,0,0,582,0,0,582,12,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,Tengah,2 BTO,Nov 2020,0,0,0,0,0,0,0,0,11,2020
343,Bishan,1 BTO,Nov 2020,0,0,0,0,0,0,0,0,11,2020
344,Tampines,1 BTO,Nov 2020,0,0,0,0,0,0,0,0,11,2020
345,Toa Payoh (Bidadari),1 BTO,Nov 2020,0,0,0,0,0,0,0,0,11,2020


In [172]:
# get subtotal of flats by month
df_mth = bto_df.groupby(['year','month'])['new_flat_offered'].sum().to_frame()
df_mth.reset_index(inplace=True)

# cast to date
df_mth['date'] = pd.to_datetime(dict(year=df_mth.year,month=df_mth.month,day=1))

# reorder the columns
df_mth = df_mth.reindex(['date','new_flat_offered'],axis=1)
df_mth.to_csv("../datasets/macro/bto_flat_monthly_cleaned.csv",index=True)

In [9]:
df_yr = bto_df.groupby(['year'])['new_flat_offered'].sum().to_frame()
df_yr.reset_index(inplace=True)
df_yr.to_csv("../datasets/macro/bto_flat_cleaned.csv",index=False)

### Supply of Private Residential Units

source: data.gov.sg

In [174]:
pte_home_df = pd.read_csv('../datasets/input/private-residential-units-launched.csv')

# split date field to year and month
pte_home_df[['year','month']] = pte_home_df['quarter'].str.split("-", 1, expand=True)

# sum the units from various segments by quarter
pte_home_df = pte_home_df.groupby(['year','month']).sum()
pte_home_df.reset_index(inplace=True)

# convert quarter to the starting month of the quarter in numeric form 
pte_home_df = pte_home_df.replace('Q1',1).replace('Q2',4).replace('Q3',7).replace('Q4',10)

# rename column
pte_home_df.rename(columns={"units":"new_pte_home_offered"},inplace=True)

pte_home_df['year'] = pte_home_df['year'].astype(np.int32)
pte_home_df['month'] = pte_home_df['month'].astype(np.int32)
pte_home_df['new_pte_home_offered'] = pte_home_df['new_pte_home_offered'].astype(np.int32)

In [175]:
df_yr = pte_home_df.copy(deep=True)

# get total residential units per year
df_yr = df_yr.groupby(['year'])['new_pte_home_offered'].sum().to_frame()
df_yr.reset_index(inplace=True)

df_yr.to_csv("../datasets/macro/new_pte_home_yr_cleaned.csv",index=False)

In [176]:
# distribute quarterly figures over months
pte_home_df = qtr_2_mth(pte_home_df)

In [177]:
pte_home_df['date'] = pd.to_datetime(dict(year=pte_home_df.year,month=pte_home_df.month,day=1))

# reorder the columns
pte_home_df = pte_home_df.reindex(['date','new_pte_home_offered'],axis=1)

In [178]:
pte_home_df

,date,new_pte_home_offered
0,2004-01-01,906
1,2004-02-01,906
2,2004-03-01,906
3,2004-04-01,1174
4,2004-05-01,1174
...,...,...
187,2019-08-01,3628
188,2019-09-01,3628
189,2019-10-01,2226
190,2019-11-01,2226


In [179]:
pte_home_df.to_csv('../datasets/macro/new_pte_home_cleaned.csv',index=False)

### Population and types of home

Source: singstat<br/>
Observation: yearly

I am mainly interested to find out if the population size affects house prices.

I will also get the number of residents living in different types of houses. 

In [180]:
p_df = pd.read_csv('../datasets/input/population_singstat.csv')

In [181]:
# Retrieve total residents and breakdown of residents for each type of house

s1 = p_df.iloc[0:1,0:].T.iloc[1:,0]
s2 = p_df.iloc[1:2,0:].T.iloc[1:,0]

pop_df = pd.DataFrame()
pop_df['year'] = s1.index
pop_df['total_residents'] = s1.values

In [182]:
pop_df.replace(',','',regex=True, inplace=True)

In [183]:
pop_df['year'] = pop_df['year'].astype(np.int32)
pop_df['total_residents'] = pop_df['total_residents'].astype(np.float).astype(np.int32)

In [184]:
pop_df.to_csv('../datasets/macro/pop_cleaned.csv',index=False)

### Number of Married People

Source: singstat<br/>
Observation: yearly

Majority of HDB flats are purchased by married couples, so it would be interestingly to see how the number of married people affect flat prices

In [185]:
m_df = pd.read_csv('../datasets/input/married_singstat.csv')

In [186]:
# transpose data from third row to a column

s = m_df.iloc[2:3,0:].T.iloc[1:,0]

married_df = pd.DataFrame()
married_df['year'] = s.index
married_df['married'] = s.values

married_df.tail()

,year,married
33,2015,"1,945,600"
34,2016,"1,974,700"
35,2017,"2,018,400"
36,2018,"2,026,800"
37,2019,"2,055,300"


In [187]:
# remove commmas from married
married_df = married_df.replace(",","",regex=True)

# change datatype of numeric columns to integer 
married_df['year'] = married_df['year'].astype(np.int32)
married_df['married'] = married_df['married'].astype(np.int64)

In [188]:
married_df.to_csv('../datasets/macro/married_cleaned.csv',index=False)

## 3. Macroeconomic factors

I will collect data from the Singapore government portals (data.gov.sg, singstat, SPIMM and associations of banks in singapore) such as CPF interest rates, Interbank interest rate, consumer price index, PMI, composite index, income, unemployment rates, GDP growth rates, etc

The observatons are monthly, quarterly and yearly. I will expand the quarterly and yearly figures to monthly.

### Composite Leading Index (CLI)
Source: data.gov.sg<br/>
Observation: quarterly

- Singapore's Composite Leading Index is used to anticipate the turning points of growth cycles, or fluctuations in the economy’s growth rate
- Expand the quarterly figures to monthly

In [64]:
cli_df = pd.read_csv('../datasets/input/composite-leading-index.csv')
cli_df.tail()

,quarter,level_1,value
164,2019-Q1,Quarterly Composite Leading Index,106.3
165,2019-Q2,Quarterly Composite Leading Index,105.0
166,2019-Q3,Quarterly Composite Leading Index,105.6
167,2019-Q4,Quarterly Composite Leading Index,104.6
168,2020-Q1,Quarterly Composite Leading Index,101.6


In [67]:
# split the date field to year and quarter
cli_df[['year','month']] = cli_df['quarter'].str.split("-", 1, expand=True)

# convert quarter to the starting month of the quarter in numeric form 
cli_df = cli_df.replace('Q1',1).replace('Q2',4).replace('Q3',7).replace('Q4',10)

# rename column
cli_df.rename(columns={"value":"cli"},inplace=True)

# reorder the columns
cli_df = cli_df.reindex(['year','month','cli'],axis=1)

cli_df['year'] = cli_df['year'].astype(np.int32)
cli_df['month'] = cli_df['month'].astype(np.int32)

cli_df = qtr_2_mth(cli_df)

cli_df['date'] = pd.to_datetime(dict(year=cli_df.year,month=cli_df.month,day=1),format='%Y%m%d')

# reorder the columns
cli_df = cli_df.reindex(['date','cli'],axis=1)

In [68]:
cli_df.head()

,date,cli
0,1978-01-01,25.8
1,1978-02-01,25.8
2,1978-03-01,25.8
3,1978-04-01,26.8
4,1978-05-01,26.8


In [70]:
cli_df.to_csv("../datasets/macro/cli_cleaned.csv",index=False)

### Perchasing Manager Index (PMI)
Source: <br/>Singapore Institute of Purchasing & Materials Management<br/>
Business Times news articles<br/><br/>
Observation: Monthly

The PMI is an indicator of business activity for the manufacturing sector, which makes up 20% of Singapore economy

In [71]:
pmi_df = pd.read_csv('../datasets/input/pmi_sipmm.csv')
pmi_df.tail()

,Month/Year,Singapore PMI,Electronics Sector PMI
16,Feb-19,50.4,49.5
17,Jan-19,50.7,49.6
18,Dec-18,51.1,49.8
19,Nov-18,51.5,49.9
20,Oct-18,51.9,50.5


In [72]:
# split the date field to year and month
pmi_df[['month','year']] = pmi_df['Month/Year'].str.split("-", 1, expand=True)

# Rename column
pmi_df.rename(columns={"Singapore PMI":"pmi"},inplace=True)

# change month to numeric form
dic = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
pmi_df.month = pmi_df.month.map(dic)

# change datatype of numeric columns to integer 
pmi_df['month'] = pmi_df['month'].astype(np.int32)
pmi_df['year'] = pmi_df['year'].astype(np.int32)

# change year to 4-digit format
pmi_df['year'] = pmi_df['year']+2000

# reorder the columns
pmi_df = pmi_df.reindex(['year','month','pmi'],axis=1)

pmi_df.sort_values(by='year',ignore_index=True,inplace=True)

pmi_df['date'] = pd.to_datetime(dict(year=pmi_df.year,month=pmi_df.month,day=1),format='%Y%m%d')

# reorder the columns
pmi_df = pmi_df.reindex(['date','pmi'],axis=1)

In [82]:
# data manually retrieved from Business Times, which publishes news about PMI every month

data= {'date':['1/1/2015','1/2/2015','1/3/2015','1/4/2015','1/5/2015','1/6/2015','1/7/2015',
               '1/8/2015','1/9/2015','1/10/2015','1/11/2015','1/12/2015','1/1/2016','1/2/2016',
               '1/3/2016','1/4/2016','1/5/2016','1/6/2016','1/7/2016','1/8/2016','1/9/2016',
               '1/10/2016','1/11/2016','1/12/2016','1/1/2017','1/2/2017','1/3/2017','1/4/2017',
               '1/5/2017','1/6/2017','1/7/2017','1/9/2017','1/9/2017','1/10/2017','1/11/2017',
               '1/12/2017','1/1/2018','1/2/2018','1/3/2018','1/4/2018','1/5/2018','1/6/2018',
               '1/7/2018','1/8/2018','1/9/2018'],
       'pmi':[49.9,49.7,49.6,49.4,59.2,50.4,49.7,49.3,48.6,48.9,49.2,49.5,49,48.5,49.4,49.8,
              49.8,49.6,49.3,49.8,50.1,50,50.2,50.6,51,50.9,51.2,51.1,50.9,50.9,51,51.8,52,
              52.6,52.9,52.8,53.1,52.7,53,52.9,52.7,52.5,52.3,52.6,52.4]}
df_news = pd.DataFrame(data)

In [93]:
df_news['date'] = pd.to_datetime(df_news['date'])
pmi_df['date'] = pd.to_datetime(pmi_df['date'])

In [95]:
df_pmi = pd.concat([df_news,pmi_df],ignore_index=True)

In [96]:
df_pmi

,date,pmi
0,2015-01-01,49.9
1,2015-01-02,49.7
2,2015-01-03,49.6
3,2015-01-04,49.4
4,2015-01-05,59.2
...,...,...
106,2020-02-01,48.7
107,2020-03-01,45.4
108,2020-04-01,44.7
109,2020-05-01,46.8


In [97]:
df_pmi.sort_values(['date'],inplace=True)

In [98]:
df_pmi

,date,pmi
0,2015-01-01,49.9
45,2015-01-01,49.9
46,2015-01-02,49.7
1,2015-01-02,49.7
2,2015-01-03,49.6
...,...,...
106,2020-02-01,48.7
107,2020-03-01,45.4
108,2020-04-01,44.7
109,2020-05-01,46.8


In [100]:
df_pmi.to_csv('../datasets/macro/pmi_cleaned.csv',index=False)

### CPF interest rates

Source: data.gov.sg<br/>
observation: monthly

When you take a housing loan from HDB, you will enjoy a concessionary interest rate. This concessionary interest rate is pegged at 0.10% above the prevailing CPF Ordinary Account (OA) interest rate, and may be adjusted in January, April, July, and October, in line with CPF interest rate revisions.[Source: HDB](https://www.hdb.gov.sg/cs/infoweb/residential/servicing-your-hdb-loan/mortgage-loan/interest-rate)

In [101]:
# import CPF interest rates
ci_df = pd.read_csv('../datasets/input/cpf-interest-rates.csv')

# housing loan interest rate is based on Ordinary account
cpf_df = ci_df[ci_df['account_type']=='Ordinary'].copy(deep=True)

# split the date field to year and quarter
cpf_df[['year','month']] = cpf_df['mth'].str.split("-", 1, expand=True)

# housing loan interest rate is based on Ordinary account
cpf_df = ci_df[ci_df['account_type']=='Ordinary'].copy(deep=True)

# split the date field to year and quarter
cpf_df[['year','month']] = cpf_df['mth'].str.split("-", 1, expand=True)

# Drop unnecessary columns
cpf_df.rename(columns={"interest_rate":"cpf_rate"},inplace=True)

# change datatypes of numeric columns to integer/float
cpf_df['year'] = cpf_df['year'].astype(np.int32)
cpf_df['month'] = cpf_df['month'].astype(np.int32)
cpf_df['cpf_rate'] = cpf_df['cpf_rate'].astype(np.float)

cpf_df['date'] = pd.to_datetime(dict(year=cpf_df.year,month=cpf_df.month,day=1))


# reorder the columns
cpf_df = cpf_df.reindex(['date','cpf_rate'],axis=1)

In [102]:
cpf_df

,date,cpf_rate
0,1955-01-01,2.5
4,1955-02-01,2.5
8,1955-03-01,2.5
12,1955-04-01,2.5
16,1955-05-01,2.5
...,...,...
3076,2019-02-01,3.5
3080,2019-03-01,3.5
3084,2019-04-01,3.5
3088,2019-05-01,3.5


In [103]:
cpf_df.to_csv('../datasets/macro/cpf_cleaned.csv', index=False)

### Consumer Price Index

source: singstat<br/>
observation: monthly

In [104]:
c_df = pd.read_csv('../datasets/input/cpi_singstat.csv')

In [105]:
s = c_df.iloc[0:1,0:].T.iloc[1:,0]

cpi_df = pd.DataFrame()
cpi_df['month'] = s.index
cpi_df['cpi'] = s.values

cpi_df.tail()

,month,cpi
708,2020 Jan,100.204
709,2020 Feb,100.279
710,2020 Mar,99.933
711,2020 Apr,99.012
712,2020 May,99.48


In [106]:
# split the date field to year and quarter
cpi_df[['year','month']] = cpi_df['month'].str.split(" ", 1, expand=True)

# change months to numeric form
dic = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
cpi_df.month = cpi_df.month.map(dic)


# change datatypes of numeric columns to integer
cpi_df['year'] = cpi_df['year'].astype(np.int32)
cpi_df['month'] = cpi_df['month'].astype(np.int32)

# cast to date
cpi_df['date'] = pd.to_datetime(dict(year=cpi_df.year,month=cpi_df.month,day=1))

# rearrange columns
cpi_df = cpi_df.reindex(['date','cpi'],axis=1)

In [107]:
cpi_df

,date,cpi
0,1961-01-01,24.542
1,1961-02-01,24.565
2,1961-03-01,24.585
3,1961-04-01,24.187
4,1961-05-01,24.053
...,...,...
708,2020-01-01,100.204
709,2020-02-01,100.279
710,2020-03-01,99.933
711,2020-04-01,99.012


In [108]:
cpi_df.to_csv('../datasets/macro/cpi_cleaned.csv',index=False)

### GDP Growth Rates

source: singstat<br/>
observation: quarterly

- Expand the quarterly figures to monthly

In [118]:
g_df = pd.read_csv('../datasets/input/gdp_growth_singstat.csv')
g_df.head()

,Variables,1976 1Q,1976 2Q,1976 3Q,1976 4Q,1977 1Q,1977 2Q,1977 3Q,1977 4Q,1978 1Q,...,2017 4Q,2018 1Q,2018 2Q,2018 3Q,2018 4Q,2019 1Q,2019 2Q,2019 3Q,2019 4Q,2020 1Q
0,GDP In Chained (2015) Dollars,8.2,7.4,7.2,7.0,6.2,7.6,6.4,7.2,5.6,...,4.5,4.8,4.9,3.0,1.1,1.0,0.2,0.7,1.0,-0.7
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
s = g_df.iloc[0:1,0:].T.iloc[1:,0]

gdp_df = pd.DataFrame()
gdp_df['month'] = s.index
gdp_df['gdp_growth'] = s.values

gdp_df.tail()

,month,gdp_growth
172,2019 1Q,1
173,2019 2Q,0.2
174,2019 3Q,0.7
175,2019 4Q,1
176,2020 1Q,-0.7


In [124]:
# split the date field to year and quarter
gdp_df[['year','month']] = gdp_df['month'].str.split(" ", 1, expand=True)

# convert quarter to the starting month of the quarter in numeric form 
gdp_df = gdp_df.replace('1Q',1).replace('2Q',4).replace('3Q',7).replace('4Q',10)

gdp_df['year'] = gdp_df['year'].astype(np.int32)
gdp_df['month'] = gdp_df['month'].astype(np.int32)

In [125]:
# reorder the columns
gdp_df = gdp_df.reindex(['year','month','gdp_growth'],axis=1)

# DO NOT RUN more than once. Otherwise it will keep expanding the rows
gdp_df = qtr_2_mth(gdp_df)
gdp_df

,year,month,gdp_growth
0,1976,1,8.2
1,1976,2,8.2
2,1976,3,8.2
3,1976,4,7.4
4,1976,5,7.4
...,...,...,...
526,2019,11,1.0
527,2019,12,1.0
528,2020,1,-0.7
529,2020,2,-0.7


In [126]:
# cast to date
gdp_df['date'] = pd.to_datetime(dict(year=gdp_df.year,month=gdp_df.month,day=1))

# reorder the columns
gdp_df = gdp_df.reindex(['date','gdp_growth'],axis=1)

In [127]:
gdp_df

,date,gdp_growth
0,1976-01-01,8.2
1,1976-02-01,8.2
2,1976-03-01,8.2
3,1976-04-01,7.4
4,1976-05-01,7.4
...,...,...
526,2019-11-01,1.0
527,2019-12-01,1.0
528,2020-01-01,-0.7
529,2020-02-01,-0.7


In [128]:
gdp_df.to_csv('../datasets/macro/gdp_cleaned.csv')

### Unemployment Rate

Source: singstat<br/>
observation: quarterly

- Expand the quarterly figures to monthly

In [129]:
u_df = pd.read_csv('../datasets/input/unemployment_singstat.csv')
u_df.head()

,Variables,1992 1Q,1992 2Q,1992 3Q,1992 4Q,1993 1Q,1993 2Q,1993 3Q,1993 4Q,1994 1Q,...,2017 4Q,2018 1Q,2018 2Q,2018 3Q,2018 4Q,2019 1Q,2019 2Q,2019 3Q,2019 4Q,2020 1Q
0,Total Unemployment Rate,1.5,2.4,1.6,1.7,1.2,2.4,1.5,1.7,1.4,...,1.8,1.9,2.7,1.9,1.9,2.0,3.0,2.0,2.0,2.2
1,Resident Unemployment Rate,1.8,2.8,2.0,2.0,1.5,2.8,1.8,2.1,1.8,...,2.5,2.7,3.9,2.5,2.6,2.8,4.2,2.8,2.7,3.1
2,Citizen Unemployment Rate,1.8,2.9,2.0,2.0,1.6,2.9,1.9,2.1,1.8,...,2.6,2.7,4.1,2.6,2.7,2.9,4.4,2.9,2.8,3.2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total population comprises residents and non r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
s = u_df.iloc[0:1,0:].T.iloc[1:,0]

unemployed_df = pd.DataFrame()
unemployed_df['month'] = s.index
unemployed_df['unemployed_rate'] = s.values

unemployed_df.tail()

,month,unemployed_rate
108,2019 1Q,2
109,2019 2Q,3
110,2019 3Q,2
111,2019 4Q,2
112,2020 1Q,2.2


In [131]:
# split the date field to year and quarter

unemployed_df[['year','month']] = unemployed_df['month'].str.split(" ", 1, expand=True)

# convert quarter to the starting month of the quarter in numeric form 

unemployed_df = unemployed_df.replace('1Q',1).replace('2Q',4).replace('3Q',7).replace('4Q',10)

# reorder the columns
unemployed_df = unemployed_df.reindex(['year','month','unemployed_rate'],axis=1)

# change datatype of numeric columns to integer
unemployed_df['year'] = unemployed_df['year'].astype(np.int32)
unemployed_df['month'] = unemployed_df['month'].astype(np.int32)

In [132]:
# DO NOT RUN more than once. Otherwise it will keep expanding the rows
unemployed_df = qtr_2_mth(unemployed_df)

# cast to date
unemployed_df['date'] = pd.to_datetime(dict(year=unemployed_df.year,month=unemployed_df.month,day=1))

# reorder the columns
unemployed_df = unemployed_df.reindex(['date','unemployed_rate'],axis=1)

unemployed_df.tail(15)

,date,unemployed_rate
324,2019-01-01,2.0
325,2019-02-01,2.0
326,2019-03-01,2.0
327,2019-04-01,3.0
328,2019-05-01,3.0
329,2019-06-01,3.0
330,2019-07-01,2.0
331,2019-08-01,2.0
332,2019-09-01,2.0
333,2019-10-01,2.0


In [133]:
unemployed_df.to_csv('../datasets/macro/unemployed_cleaned.csv',index=False)

### SIBOR 

Source: abs.org.sg<br/>
Observation : business days

- Housing loans are commonly pegged to SIBOR (Singapore Interbank Offered Rate). There are several types of SIBOR, and the only one used for HDB flat loan is 3-month.
- Calculate average the rates for each month

In [136]:
sibor_df = pd.read_csv('../datasets/input/sibor_abs.csv')
sibor_df

,SIBOR DATE,SIBOR 1M,SIBOR 3M,SIBOR 6M,SIBOR 12M
0,2/1/2014,0.35369,0.40267,0.48191,0.59728
1,3/1/2014,0.35057,0.40267,0.48191,0.59728
2,6/1/2014,0.35520,0.40361,0.48191,0.59728
3,7/1/2014,0.35520,0.40361,0.48191,0.59728
4,8/1/2014,0.35520,0.40461,0.48191,0.59728
...,...,...,...,...,...
1598,22/5/2020,0.24850,0.55916,0.74725,1.02908
1599,26/5/2020,0.24450,0.55091,0.74725,1.02908
1600,27/5/2020,0.24650,0.55916,0.74725,1.03013
1601,28/5/2020,0.24900,0.55941,0.74725,1.03013


In [137]:
# split the date to day, month, year
sibor_df[['day','month','year']] = sibor_df['SIBOR DATE'].str.split("/", 2, expand=True)

# convert numeric columns to integer
sibor_df['month'] = sibor_df['month'].astype(np.int32)
sibor_df['year'] = sibor_df['year'].astype(np.int32)

In [139]:
# store sibor_3m average in a temp dataframe
sibor_df = sibor_df.groupby(['year','month'])['SIBOR 3M'].mean().reset_index()

In [140]:
sibor_df

,year,month,SIBOR 3M
0,2014,1,0.404570
1,2014,2,0.402225
2,2014,3,0.403190
3,2014,4,0.403825
4,2014,5,0.403760
...,...,...,...
72,2020,1,1.750963
73,2020,2,1.710662
74,2020,3,1.205994
75,2020,4,0.969687


In [141]:
# rename column to lowercase and replace space with underscore
sibor_df.rename(columns={'SIBOR 3M':'sibor_3m'},inplace=True)

# cast to date
sibor_df['date'] = pd.to_datetime(dict(year=sibor_df.year,month=sibor_df.month,day=1))

# reorder the columns
sibor_df = sibor_df.reindex(['date','sibor_3m'],axis=1)
sibor_df

,date,sibor_3m
0,2014-01-01,0.404570
1,2014-02-01,0.402225
2,2014-03-01,0.403190
3,2014-04-01,0.403825
4,2014-05-01,0.403760
...,...,...
72,2020-01-01,1.750963
73,2020-02-01,1.710662
74,2020-03-01,1.205994
75,2020-04-01,0.969687


In [142]:
sibor_df.to_csv('../datasets/macro/sibor_cleaned.csv',index=False)

### Monthly Income

source: singstat<br/>
observation: yearly


I will get the median income of the population of each year and see how it affects house prices

In [143]:
ic_df = pd.read_csv('../datasets/input/monthly_income_singstat.csv')
ic_df.head()

,Variables,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Median (50th Percentile),"2,387","2,380","2,410","2,326",-,"2,449","2,543","2,897","2,927","3,000","3,249","3,480","3,705","3,770","3,949","4,056","4,232","4,437","4,563"
1,20th Percentile,"1,383","1,346","1,334","1,305",-,"1,286","1,356","1,489","1,500","1,600","1,733","1,740","1,885","1,972","2,012","2,106","2,200","2,340","2,457"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data are for mid-year. As the income data are...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
# transpose first row of dataframe to get median monthly income

s = ic_df.iloc[0:1,0:].T.iloc[1:,0]

income_df = pd.DataFrame()
income_df['year'] = s.index
income_df['mth_income'] = s.values
income_df.head()

,year,mth_income
0,2001,"2,387"
1,2002,"2,380"
2,2003,"2,410"
3,2004,"2,326"
4,2005,-


In [145]:
# There is a missing value for income in 2005
# i will get the average of incomes for 2004 and 2006

# remove commas from income
income_df = income_df.replace(",","",regex=True)

# get the incomes 
inc_2004 = int(income_df[income_df['year']=='2004'].values[0][1])
inc_2006 = int(income_df[income_df['year']=='2006'].values[0][1])

# missing value for 2005 - update dataframe with the average
income_df.iloc[4,1] = (inc_2004+inc_2006)/2

In [71]:
income_df

,year,mth_income
0,2001,2387
1,2002,2380
2,2003,2410
3,2004,2326
4,2005,2387.5
5,2006,2449
6,2007,2543
7,2008,2897
8,2009,2927
9,2010,3000


In [146]:
# change datatype of numeric columns to integer 
income_df['year'] = income_df['year'].astype(np.int32)
income_df['mth_income'] = income_df['mth_income'].astype(np.int32)

In [147]:
income_df.to_csv('../datasets/macro/income_cleaned.csv',index=False)

### HDB Resale Price Index

Source: singstat<br/>
observation : quarterly

- Expand the quarterly figures to monthly

In [158]:
hi_df = pd.read_csv('../datasets/input/hdb_price_index_singstat.csv')
hi_df.head()

,Variables,1990 1Q,1990 2Q,1990 3Q,1990 4Q,1991 1Q,1991 2Q,1991 3Q,1991 4Q,1992 1Q,...,2017 4Q,2018 1Q,2018 2Q,2018 3Q,2018 4Q,2019 1Q,2019 2Q,2019 3Q,2019 4Q,2020 1Q
0,Total,24.3,24.4,25.0,24.7,24.9,25.5,25.2,25.1,25.8,...,132.6,131.6,131.7,131.6,131.4,131.0,130.8,130.9,131.5,131.5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The index is based on quarterly average resale...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# transpose first row of dataframe
s = hi_df.iloc[0:1,0:].T.iloc[1:,0]
hdb_index_df = pd.DataFrame()
hdb_index_df['month'] = s.index
hdb_index_df['hdb_index'] = s.values

# split the month into year and month
hdb_index_df[['year','month']] = hdb_index_df['month'].str.split(" ", 1, expand=True)

# convert quarters to the first month of the quarter
hdb_index_df = hdb_index_df.replace('1Q',1).replace('2Q',4).replace('3Q',7).replace('4Q',10)

# convert numeric columns to integer
hdb_index_df['year'] = hdb_index_df['year'].astype(np.int32)
hdb_index_df['month'] = hdb_index_df['month'].astype(np.int32)

In [160]:
# reorder the columns
hdb_index_df = hdb_index_df.reindex(['year','month','hdb_index'],axis=1)

In [161]:
# DO NOT RUN more than once. Otherwise it will keep expanding the rows
hdb_index_df = qtr_2_mth(hdb_index_df)

hdb_index_df['date'] = pd.to_datetime(dict(year=hdb_index_df.year,month=hdb_index_df.month,day=1))

# reorder the columns
hdb_index_df = hdb_index_df.reindex(['date','hdb_index'],axis=1)

In [162]:
hdb_index_df.tail(10)

,date,hdb_index
353,2019-06-01,130.8
354,2019-07-01,130.9
355,2019-08-01,130.9
356,2019-09-01,130.9
357,2019-10-01,131.5
358,2019-11-01,131.5
359,2019-12-01,131.5
360,2020-01-01,131.5
361,2020-02-01,131.5
362,2020-03-01,131.5


In [163]:
hdb_index_df.to_csv('../datasets/macro/hdb_index_cleaned.csv',index=False)

### Private property price index

Source: data.gov.sg<br/>
observation: quarterly

- Expand the quarterly figures to monthly

In [164]:
pi_df = pd.read_csv('../datasets/input/private-residential-property-price-index.csv') 
pi_df.tail()

,quarter,property_type,index
535,2019-Q3,Landed,165.8
536,2019-Q3,Non-Landed,150.0
537,2019-Q4,All Residential,153.6
538,2019-Q4,Landed,171.8
539,2019-Q4,Non-Landed,149.6


In [166]:
# split into condo and all private residential

condo_index_df = pi_df[pi_df['property_type']=='Non-Landed'].copy(deep=True)
pte_index_df = pi_df[pi_df['property_type']=='All Residential'].copy(deep=True)

# split quarter into year and month 
condo_index_df[['year','month']] = condo_index_df['quarter'].str.split("-", 1, expand=True)
pte_index_df[['year','month']] = pte_index_df['quarter'].str.split("-", 1, expand=True)

# rename column to meaningful name
condo_index_df.rename(columns={"index":"condo_index"},inplace=True)
pte_index_df.rename(columns={"index":"pte_index"},inplace=True)

# preparing for expanding quarterly figures to monthly
condo_index_df = condo_index_df.replace('Q1',1).replace('Q2',4).replace('Q3',7).replace('Q4',10)
pte_index_df = pte_index_df.replace('Q1',1).replace('Q2',4).replace('Q3',7).replace('Q4',10)

# convert numeric columns to integer
condo_index_df['year'] = condo_index_df['year'].astype(np.int32)
condo_index_df['month'] = condo_index_df['month'].astype(np.int32)
pte_index_df['year'] = pte_index_df['year'].astype(np.int32)
pte_index_df['month'] = pte_index_df['month'].astype(np.int32)

# reorder the columns
condo_index_df = condo_index_df.reindex(['year','month','condo_index'],axis=1)
pte_index_df = pte_index_df.reindex(['year','month','pte_index'],axis=1)

In [167]:
# Expand quarterly figures to monthly
# DO NOT RUN more than once. Otherwise it will keep expanding the rows
condo_index_df = qtr_2_mth(condo_index_df)
pte_index_df = qtr_2_mth(pte_index_df)

condo_index_df['date'] = pd.to_datetime(dict(year=condo_index_df.year,month=condo_index_df.month,day=1))
pte_index_df['date'] = pd.to_datetime(dict(year=pte_index_df.year,month=pte_index_df.month,day=1))

# reorder the columns
condo_index_df = condo_index_df.reindex(['date','condo_index'],axis=1)
pte_index_df = pte_index_df.reindex(['date','pte_index'],axis=1)

In [168]:
condo_index_df.to_csv('../datasets/macro/condo_index_cleaned.csv',index=False)
pte_index_df.to_csv('../datasets/macro/pte_index_cleaned.csv',index=False)

## 4. Points of Interest

### Primary and Secondary Schools

Source: data.gov.sg

- fetch the geocodes of schools based on their physical addresses

In [52]:
school_df = pd.read_csv('../datasets/input/general-information-of-schools.csv')

school_df = school_df[['school_name','address','mainlevel_code']].copy(deep=True)

school_df['school_name'] = school_df['school_name'].str.lower()
school_df['address'] = school_df['address'].str.lower()
school_df['mainlevel_code'] = school_df['mainlevel_code'].str.lower()

# add columns for use in feature engineering 
school_df['lat'] = 0
school_df['lng'] = 0

school_df = school_df.reindex(['school_name','lat','lng','address','mainlevel_code'],axis=1)
school_df.to_csv('../datasets/poi/school_cleaned.csv', index = False)

### Hawker Centres and Markets

Source: data.gov.sg

In [2]:
import xml.etree.ElementTree as ET
filePath = r'../datasets/input/hawker-centres-kml.kml'
tree = ET.parse(filePath) 
root = tree.getroot()

hc = []
lat=[]
lng=[]

for Document in root:
    for Folder in Document:
        for Placemark in Folder:
            for ExtendedData in Placemark:
                for SchemaData in ExtendedData:
                    for SimpleData in SchemaData:
                        if SimpleData.get("name") == "NAME":
                            hc.append(SimpleData.text)
                        if SimpleData.get("name") == "LATITUDE":
                            lat.append(SimpleData.text)
                        if SimpleData.get("name") == "LONGITUDE":
                            lng.append(SimpleData.text)
    #print(len(lat),len(lng),len(hc))
    
dic = {'hawker_centre':hc,'lat':lat,'lng':lng}
hc_df = pd.DataFrame.from_dict(dic)

In [3]:
hc_df.isna().sum()

hawker_centre    0
lat              0
lng              0
dtype: int64

In [18]:
hc_df['lat'] = hc_df['lat'].astype(float)

In [16]:
hc_df['lng'] = hc_df['lng'].astype(float)

In [19]:
hc_df.eq(0).any().any()

True

In [20]:
hc_df[hc_df['lat']==0]

,hawker_centre,lat,lng
49,Fernvale Hawker Centre,0.0,0.0
50,Punggol Town Hub Hawker Centre,0.0,0.0
51,Bukit Batok Hawker Centre,0.0,0.0
52,Bukit Canberra Hawker Centre,0.0,0.0
53,Punggol Digital District Hawker Centre,0.0,0.0
59,Dawson Hawker Centre,0.0,0.0
60,Woodleigh Village Hawker Centre,0.0,0.0
67,Anchorvale Village Hawker Centre,0.0,0.0
68,Market Street Hawker Centre,0.0,0.0
69,Bukit Panjang North Hawker Centre,0.0,0.0


In [24]:
# All the hawker centres with zero for lat and lng have NOT been built and so will NOT be saved

hc_df[hc_df['lat'] != 0].to_csv('../datasets/poi/hawker_centre_cleaned.csv', index = False)

### MRT and LRT Stations

Source: https://www.kaggle.com/yxlee245/singapore-train-station-coordinates?select=mrt_lrt_data.csv

In [92]:
station_df = pd.read_csv('../datasets/input/datasets_287088_590207_mrt_lrt_data.csv')

In [94]:
station_df = station_df.reindex(['station_name','lat','lng','type'],axis=1)

In [95]:
station_df

,station_name,lat,lng,type
0,Jurong East,1.333207,103.742308,MRT
1,Bukit Batok,1.349069,103.749596,MRT
2,Bukit Gombak,1.359043,103.751863,MRT
3,Choa Chu Kang,1.385417,103.744316,MRT
4,Yew Tee,1.397383,103.747523,MRT
...,...,...,...,...
152,Punggol Point,1.416932,103.906680,LRT
153,Samudera,1.415955,103.902185,LRT
154,Nibong,1.411865,103.900321,LRT
155,Sumang,1.408501,103.898605,LRT


In [103]:
station_df.to_csv('../datasets/poi/mrt_cleaned.csv',index=False)

### Shopping Malls

Source:
https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore<br/>
https://thenewageparents.com/list-of-shopping-malls-in-singapore/


There is no single website that lists all the shopping malls so I scraped from 2 websites. There are duplicates that will be removed.

**Note:** The data on the websites can change. I notice the wikipedia list changed when I scraped again a few days after my  first attempt.

- use Beautiful Soup to scrap the data from web page

In [53]:
# First website 
# Use the requests library to get the html from first website
res = requests.get('https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore')

# Create a soup object from the html
soup = BeautifulSoup(res.content, 'lxml')

malls = soup.find('div',{'class':'mw-parser-output'})

# create a new dataframe
df_malls = pd.DataFrame(columns=['mall','lat','lng'])

# Loop through each <a> tag to get mall's name and its geocode
idx = 0
for m in malls.find_all('a', {'class': 'new'}):
    df_malls.loc[idx] = [m.text,0,0]
    
    idx += 1

df_malls

,mall,lat,lng
0,100 AM,0,0
1,313@Somerset,0,0
2,Aperia,0,0
3,Balestier Hill Shopping Centre,0,0
4,Bugis Cube,0,0
...,...,...,...
65,Fernvale Point,0,0
66,Fitzpatrick's,0,0
67,Serangoon Plaza,0,0
68,Northshore Plaza,0,0


In [424]:
# Fitzpatrick has ceased operation many years ago and so will be removed
df_malls.drop(index=66,inplace=True)


# There is another non-existent mall that was gone when I scraped a few days after the first attempt,
# so Fitzpatrick may also be removed from the website by the time you scrape

In [54]:
# Second website
# Use the requests library to get the html from second website
res = requests.get('https://thenewageparents.com/list-of-shopping-malls-in-singapore/')

# Create a soup object from the html
soup = BeautifulSoup(res.content, 'lxml')

malls = soup.find('div',{'class':'post-content'})

for m in malls.find_all('span'):
    
    if not ("www.fb.com" in m.text):
        df_malls.loc[idx] = [m.text,0,0]
        
    idx += 1

df_malls

,mall,lat,lng
0,100 AM,0,0
1,313@Somerset,0,0
2,Aperia,0,0
3,Balestier Hill Shopping Centre,0,0
4,Bugis Cube,0,0
...,...,...,...
215,Zhongshan Mall,0,0
217,I12 Katong,0,0
219,313 Somerset,0,0
221,support@thenewageparents.com,0,0


In [55]:
# footer info was added so need to be removed
df_malls.drop(index=221,inplace=True)
df_malls.drop(index=222,inplace=True)

# remove the second "313 Somerset". It was listed as "313@Somerset" at the top
df_malls.drop(index=219,inplace=True)

In [56]:
# remove other duplicates

df_malls.drop_duplicates(inplace=True)
df_malls.reset_index(drop=True,inplace=True)

In [57]:
df_malls.to_csv('../datasets/poi/mall_cleaned.csv', index=False)

### Nature Parks

Source: data.gov.sg

- scrape the data from the geojson file downloaded

In [ ]:
# create a new dataframe for nature parks
df_parks = pd.DataFrame(columns=['park','lat','lng'])

with open('../datasets/input/parks-geojson.geojson') as f:
    data = json.load(f)

i=0
for feature in data['features']:
    
    soup = BeautifulSoup(feature['properties']['Description'], 'lxml')
    
    table = soup.find('table')
       
    for line in table.findAll('tr'):
        
        name = re.findall(r"\bNAME\b",line.getText())
        
        if len(name)>0:
            park = re.findall(r"NAME\s([\S\s]*)",line.getText())
            
            df_parks.loc[i] = [park[0],feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0]]
            
            i += 1


In [ ]:
df_parks.to_csv('../datasets/poi/park_cleaned.csv', index=False)

### Crematoria, Columbaria and Cemeteries

source: data.gov.sg

- scrape the data from the geojson file downloaded

In [124]:

# create a new dataframe for crematoria
df_crematoria = pd.DataFrame(columns=['name','lat','lng','type'])

with open('../datasets/input/crematoria-geojson.geojson') as f:
    data = json.load(f)

i=0
for feature in data['features']:
    
    soup = BeautifulSoup(feature['properties']['Description'], 'lxml')
    
    table = soup.find('table')
       
    for line in table.findAll('tr'):
        
        name = re.findall(r"\bNAME\b",line.getText())
        
        if len(name)>0:
            crema = re.findall(r"NAME\s([\S\s]*)",line.getText())
            
            df_crematoria.loc[i] = [crema[0],feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0],'crematorium']
            
            i += 1


In [125]:
# create a new dataframe for crematoria
df_columbaria = pd.DataFrame(columns=['name','lat','lng','type'])

with open('../datasets/input/dedicated-columbaria-geojson.geojson') as f:
    data = json.load(f)

i=0
for feature in data['features']:
    
    soup = BeautifulSoup(feature['properties']['Description'], 'lxml')
    
    table = soup.find('table')
       
    for line in table.findAll('tr'):
        
        name = re.findall(r"\bNAME\b",line.getText())
        
        if len(name)>0:
            colum = re.findall(r"NAME\s([\S\s]*)",line.getText())
            
            df_columbaria.loc[i] = [colum[0],feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0],'columbaria']
            
            i += 1


In [121]:
# create a new dataframe for cemeteries
df_cemeteries = pd.DataFrame(columns=['name','lat','lng','type'])

with open('../datasets/input/active-cemeteries-geojson.geojson') as f:
    data = json.load(f)

i=0
for feature in data['features']:
    
    soup = BeautifulSoup(feature['properties']['Description'], 'lxml')
    
    table = soup.find('table')
       
    for line in table.findAll('tr'):
        
        name = re.findall(r"\bNAME\b",line.getText())
        
        if len(name)>0:
            cem = re.findall(r"NAME\s([\S\s]*)",line.getText())
            
            df_cemeteries.loc[i] = [cem[0],feature['geometry']['coordinates'][1],feature['geometry']['coordinates'][0],'cemetery']
            
            i += 1



In [122]:
df_cemeteries.head()

,name,lat,lng,type
0,Chua Chu Kang Ahmadiyya Jama'at Cemetery,1.369400,103.688042,cemetery
1,Chua Chu Kang Bahai Cemetery,1.374913,103.692977,cemetery
2,Chua Chu Kang Chinese Cemetery,1.381858,103.686390,cemetery
3,Chua Chu Kang Christian Cemetery,1.373497,103.689689,cemetery
4,Chua Chu Kang Hindu Cemetery,1.369422,103.685886,cemetery


In [128]:
df_after_death = pd.concat([df_cemeteries,df_columbaria,df_crematoria],ignore_index=True)

In [129]:
df_after_death

,name,lat,lng,type
0,Chua Chu Kang Ahmadiyya Jama'at Cemetery,1.369400,103.688042,cemetery
1,Chua Chu Kang Bahai Cemetery,1.374913,103.692977,cemetery
2,Chua Chu Kang Chinese Cemetery,1.381858,103.686390,cemetery
3,Chua Chu Kang Christian Cemetery,1.373497,103.689689,cemetery
4,Chua Chu Kang Hindu Cemetery,1.369422,103.685886,cemetery
5,Chua Chu Kang Jewish Cemetery,1.371647,103.699983,cemetery
6,Chua Chu Kang Muslim Cemetery,1.383451,103.687554,cemetery
7,Chua Chu Kang Parsi Cemetery,1.371738,103.699554,cemetery
8,Lawn Cemetery,1.373497,103.689689,cemetery
9,"State Cemetery, Kranji",1.419480,103.757139,cemetery


In [130]:
df_after_death.to_csv('../datasets/poi/after_death_cleaned.csv', index=False)

### Sports Facilities

Source: data.gov.sg

- scrape the data from the geojson file provided
- convert categorical data to dummy variables

In [ ]:
# create a new dataframe for sports facilities
df_sports = pd.DataFrame(columns=['road','facilities','lat','lng'])

# Create an empty list to hold all facilities found
facilities_list = []

with open('../datasets/input/sportsg-sport-facilities-geojson.geojson') as f:
    data = json.load(f)

i=0
for feature in data['features']:
    
    soup = BeautifulSoup(feature['properties']['Description'], 'lxml')
    
    table = soup.find('table')
       
    for line in table.findAll('tr'):
        
        
        fa = re.findall(r"\bFACILITIES\b",line.getText())
        if len(fa)>0:
            fac = re.findall(r"FACILITIES\s([\S\s]*)",line.getText())
            
            # add to facilities_list
            facilities_list.extend(fac[0].lower().strip().split('/'))
        
        
        rd = re.findall(r"\bROAD_NAME\b",line.getText())
        if len(rd)>0:
            road = re.findall(r"ROAD_NAME\s([\S\s]*)",line.getText())
            
            #print(road[0])
            
            
            gp = feature['geometry']['coordinates']

        
    if len(fac)>0 and len(road)>0 and g[0][1]>0:
        df_sports.loc[i] = [road[0],fac[0].lower().strip(),gp[0][0][1],gp[0][0][0]]

    i += 1

In [ ]:
# Now facilities_list has collected all types of facilities, many of them are duplicates
# So use set to extract unique types
f_list=[]
for i in facilities_list:
    f_list.append(i.strip())
    
facilities_set = set(f_list)
print(facilities_set)

In [ ]:
# Now create the dummy variables, one for each type of facility
for i in facilities_set:
    df_sports[i] = 0

for index, row in df_sports.iterrows():
    f_list = row['facilities'].strip().split('/') # split the facilities string
    
    for j in f_list:
        #set corresponding column to 1
        if j in facilities_set:
            df_sports.loc[index,j]=1

In [ ]:
df_sports

In [ ]:
df_sports.rename(columns={'gateball & petanque courts':'gateball_petanque_courts',
                'swimming complex':'swimming_complex','squash centre':'squash_centre',
                'netball centre':'netball_centre','lawn bowl':'lawn_bowl','sports hall':'sports_hall',
                'futsal court':'futsal_court','tennis centre':'tennis_centre',
                'practice track':'practice_track','hockey pitch':'hockey_pitch'
                },inplace=True)

In [ ]:
df_sports.drop(labels='facilities',axis=1,inplace=True)

In [ ]:
df_sports.head()

In [ ]:
df_sports.to_csv('../datasets/poi/sport_cleaned.csv', index=False)

### Attractions

In [573]:
import xml.etree.ElementTree as ET
filePath = r'../datasets/input/TOURISM.kml'
tree = ET.parse(filePath) 
placemarks = tree.findall('.//{http://www.opengis.net/kml/2.2}Placemark')



attractions = []
lat = []
lng = []

for attributes in placemarks:
    for subAttribute in attributes:
        for point in subAttribute:
            if point.tag == '{http://www.opengis.net/kml/2.2}coordinates':
                geocode = point.text.split(',')
                lat.append(geocode[1])
                lng.append(geocode[0].strip())
                
                
        if subAttribute.tag == '{http://www.opengis.net/kml/2.2}description':        
            soup = BeautifulSoup(subAttribute.text, 'html')
    
            table = soup.find('table')

            i=0
            for tr in table.findAll('tr'):
                i+=1
                if i==8:
                    j=0
                    for td in tr.findAll('td'):
                        j+=1
                        if j==2:
                            attractions.append(td.getText())
                            
    #print(lat,lng,attractions)
    dic = {'attraction':attractions,'lat':lat,'lng':lng}
    df_attractions = pd.DataFrame.from_dict(dic)

In [574]:
df_attractions

,attraction,lat,lng
0,"Chinatown Heritage Centre, Singapore",1.283510000442852,103.8443500000813
1,"Thian Hock Keng Temple, Singapore",1.280939999723204,103.847629999838
2,Eurasian Heritage Centre: Singapore Attraction,1.310070000376672,103.8994200001141
3,Shophouses,1.277218600264318,103.8373355996684
4,Pinnacle@Duxton,1.275490000070019,103.8414200003462
...,...,...,...
101,"Kusu Island, Singapore",1.2230964997547,103.860523199782
102,"Jurong Bird Park, Singapore: Attractions & Thi...",1.318706000436972,103.7064420000953
103,HortPark,1.279069000226187,103.7997239999188
104,Adventure Cove Waterpark™ Singapore,1.257338799985209,103.8195254000482


In [ ]:
#df_attractions.to_csv("../datasets/poi/attraction_cleaned.csv",index=False)

### Libraries

source: data.gov.sg

In [98]:
import xml.etree.ElementTree as ET

lng = []
lat = []

filePath = r'../datasets/input/libraries-kml.kml'
tree = ET.parse(filePath) 
placemarks = tree.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

for attributes in placemarks:
    for subAttribute in attributes:
        for point in subAttribute:
            if point.tag == '{http://www.opengis.net/kml/2.2}coordinates':
                geocode = point.text.split(',')
                lat.append(geocode[1])
                lng.append(geocode[0].strip())
                #print(len(lat),len(lng))   
                


In [99]:
import xml.etree.ElementTree as ET
filePath = r'../datasets/input/libraries-kml.kml'
tree = ET.parse(filePath) 
root = tree.getroot()

libraries = []
lat=[]
lng=[]

for Document in root:
    for Folder in Document:
        for Placemark in Folder:
            for Point in Placemark:
                for coordinates in Point:
                    if point.tag == '{http://www.opengis.net/kml/2.2}coordinates':
                        if not '\n' in coordinates.text:
                            geocode = coordinates.text.split(',')
                            lat.append(geocode[1])
                            lng.append(geocode[0])
    #print(len(lat),len(lng))


for Document in root:
    for Folder in Document:
        for Placemark in Folder:
            for ExtendedData in Placemark:
                for SchemaData in ExtendedData:
                    for SimpleData in SchemaData:
                        if SimpleData.get("name") == "NAME":
                            libraries.append(SimpleData.text)
    #print(len(libraries))
    
    dic = {'library':libraries,'lat':lat,'lng':lng}
    df_libraries = pd.DataFrame.from_dict(dic)

In [100]:
df_libraries

,library,lat,lng
0,Serangoon Public Library,1.35072709374027,103.872256917786
1,library@orchard,1.30040819039694,103.839669047112
2,Geylang East Public Library,1.31744297942454,103.885837273218
3,Jurong West Public Library,1.34063599253795,103.704512325533
4,Bukit Batok Public Library,1.34949754800173,103.749285191724
5,Sengkang Public Library,1.39132876236971,103.894688239041
6,Marine Parade Public Library,1.30466966872275,103.909581446984
7,Pasir Ris Public Library,1.37235750960519,103.949439880186
8,library@chinatown,1.28526073700342,103.844942912669
9,library@esplanade,1.28941165551051,103.855770250108


In [101]:
df_libraries.to_csv('../datasets/poi/library_cleaned.csv',index=False)